In [1]:
pip install bctpy jgf

     |████████████████████████████████| 102 kB 5.2 MB/s eta 0:00:01
  Created wheel for jgf: filename=jgf-0.2.2-py3-none-any.whl size=10821 sha256=6869d2f4067e89a83bb8351078bf9be0d257b250792b0669167925db7a1c83cd
  Stored in directory: /home/jovyan/.cache/pip/wheels/73/d9/37/4c2e9282bc99f955151b06d0b7b970c521aef95f484084fc13
Successfully built jgf
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os,sys,glob
from matplotlib import colors as mcolors
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import seaborn as sns
from itertools import combinations
from sklearn.metrics import mean_squared_error
from scipy import stats
import bct
import jgf

In [67]:
## this will add a subjectID and sessionID column to the output data
def addSubjectsSessions(subject,session,path,data):
    
    if 'subjectID' not in data.keys():
        data['subjectID'] = [ str(subject) for f in range(len(data)) ]
    
    if 'sessionID' not in data.keys():
        data['sessionID'] = [ str(session) for f in range(len(data)) ]
        
    return data

## this function calles checkForDuplicates and attempts to find duplicates. then uses that output, sets a dumby sessionID if not present,
## and appends the object data
def appendData(subjects,sessions,paths,finish_dates,obj,filename):
        
    # check for duplicates. if so, remove
    finish_dates, subjects, sessions, paths = checkForDuplicates(obj,finish_dates,subjects,sessions,paths)

    # append data to appropriate lists
    subjects = np.append(subjects,obj['output']['meta']['subject'])
    if 'session' in obj['output']['meta'].keys():
        sessions = np.append(sessions,obj['output']['meta']['session'])
    else:
        sessions = np.append(sessions,'1')
    paths = np.append(paths,"input/"+obj["path"]+"/"+filename)
    finish_dates = np.append(finish_dates,obj['finish_date'])
    
    return finish_dates, subjects, sessions, paths

## this function will call addSubjectsSessions to add the appropriate columns and will append the object data to a study-wide dataframe
def compileData(paths,subjects,sessions,data):
    # loops through all paths
    for i in range(len(paths)):
        # if network, load json. if not, load csv
        if '.json.gz' in paths[i]:
            tmpdata = pd.read_json(paths[i],orient='index').reset_index(drop=True)
            tmpdata = addSubjectsSessions(subjects[i],sessions[i],paths[i],tmpdata)
        else:
            tmpdata = pd.read_csv(paths[i])
            tmpdata = addSubjectsSessions(subjects[i],sessions[i],paths[i],tmpdata)

        data = data.append(tmpdata,ignore_index=True)

    # replace empty spaces with nans
    data = data.replace(r'^\s+$', np.nan, regex=True)
    
    return data

def compileNetworkAdjacencyMatrices(paths,subjects,sessions,data):
    
    # loop through paths and append adjacency matrix to dictionary
    for i in range(len(paths)):
        data[subjects[i]+'_sess'+sessions[i]] = jgf.conmat.load(paths[i],compressed=True)[0]

    return data

### subjects dataframe generation
## this function will make a dataframe from a list of subjects and groups. will also add a color column for easy plotting
def collectSubjectData(topPath,dataPath,groups,subjects,colors):

    # set up variables
    data_columns = ['subjectID','classID','colors']
    data =  pd.DataFrame([],columns=data_columns)

    # populate structure
    data['subjectID'] = [ f for g in groups for f in subjects[g] ]
    data['classID'] = [ g for g in groups for f in range(len(subjects[g]))]
    data['colors'] = [ colors[c] for c in colors for f in subjects[c]]

    # output data structure for records and any further analyses
    if not os.path.exists(dataPath):
        os.mkdir(dataPath)

    data.to_csv(dataPath+'subjects.csv',index=False)

    return data

## this will create a subject-specific color for each subject in the subjects dataframe
def createColorDictionary(data,measure,colorPalette):

    # Create subject keys and color values
    keys = data[measure].unique()
    values = sns.color_palette(colorPalette,len(keys))
    values = values.as_hex()

    # zip dictionary together
    colors_dict = dict(zip(keys,values))

    return colors_dict

### load data
## this function is useful for identifying duplicate datatypes. if it finds one, it will update the data with the latest finishing dataset.
def checkForDuplicates(obj,finish_dates,subjects,sessions,paths):
    
    # first checks if there is a session id available in the keys of the object. if finds one, then checks if the subject and session ID 
    # were already looped over. if so, will delete position in list and update with appropriate path. if it doesn't find a session ID, it
    # just attempts to find if the subject has already been looped over
    if 'session' in obj['output']['meta'].keys():
        if (obj['output']['meta']['subject'] in subjects) and (obj['output']['meta']['session'] in sessions):
            index = np.where(np.logical_and(subjects == obj['output']['meta']['subject'],sessions == obj['output']['meta']['session']))
            if finish_dates[index] <= obj["finish_date"]:
                subjects = np.delete(subjects,index)
                paths = np.delete(paths,index)
                sessions = np.delete(sessions,index)
                finish_dates = np.delete(finish_dates,index)
    else:
        if (obj['output']['meta']['subject'] in subjects):
            index = np.where(subjects == obj['output']['meta']['subject'])
            if finish_dates[index] <= obj["finish_date"]:
                subjects = np.delete(subjects,index)
                paths = np.delete(paths,index)
                sessions = np.delete(sessions,index)
                finish_dates = np.delete(finish_dates,index)

    return finish_dates, subjects, sessions, paths

## this function is the wrapper function that calls all the prevouis functions to generate a dataframe for the entire project of the appropriate datatype
def collectData(datatype,datatype_tags,tags,filename,outPath,net_adj):

    import requests
    import pandas as pd

    # grab path and data objects
    objects = requests.get('https://brainlife.io/api/warehouse/secondary/list/%s'%os.environ['PROJECT_ID']).json()
    
    # subjects and paths
    subjects = []
    sessions = []
    paths = []
    finish_dates = []
    
    # set up output
    data = pd.DataFrame()

    # loop through objects and find appropriate objects based on datatype, datatype_tags, and tags. can include drop tags ('!'). this logic could probably be simplified
    for obj in objects:
        if obj['datatype']['name'] == datatype:
            # if datatype_tags is set, identify data using this info. if not, just use tag data. if no tags either, just append if meets datatype criteria. will check for filter with a not tag (!)
            if datatype_tags:
                # if the input datatype_tags are included in the object's datatype_tags, look for appropriate tags. if no tags, just append
                if set(datatype_tags).issubset(obj['output']['datatype_tags']):
                    # if tags is set, identify the data using this info
                    if tags:
                        # if input tags are included in object's tags, append. check if user wants to filter with a not tag (!)
                        if set(tags).issubset(obj['output']['tags']):
                            finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
                        elif '!' in str(tags):
                            tag = [ f for f in tags if '!' in str(f) ]
                            tag_drop = [ f for f in tags if f not in tag ]
                            if not set([ f.replace('!','') for f in tag]).issubset(obj['output']['tags']):
                                if set(tag_drop).issubset(obj['output']['tags']):
                                    finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
                    else:
                        finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
                elif '!' in str(datatype_tags):
                    datatype_tag = [ f for f in datatype_tags if '!' in str(f) ]
                    datatype_tag_drop = [ f for f in datatype_tags if f not in datatype_tag ]
                    if not set([ f.replace('!','') for f in datatype_tag]).issubset(obj['output']['datatype_tags']):
                        if tags:
                            if set(tags).issubset(obj['output']['tags']):
                                finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
                            elif '!' in str(tags):
                                tag = [ f for f in tags if '!' in str(f) ]
                                tag_drop = [ f for f in tags if f not in tag ]
                                if not set([ f.replace('!','') for f in tag]).issubset(obj['output']['tags']):
                                    if set(tag_drop).issubset(obj['output']['tags']):
                                        finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
                        else:
                            finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
            else:
                if tags:
                    if set(tags).issubset(obj['output']['tags']):
                        finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
                    elif '!' in str(tags):
                        tag = [ f for f in tags if '!' in str(f) ]
                        tag_drop = [ f for f in tags if f not in tag ]
                        if not set([ f.replace('!','') for f in tag]).issubset(obj['output']['tags']):
                            if set(tag_drop).issubset(obj['output']['tags']):
                                finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
                else:
                    finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)

    # shuffle data so subjects are in order
    paths = [z for _,_,z in sorted(zip(subjects,sessions,paths))]
    subjects = [x for x,_,_ in sorted(zip(subjects,sessions,paths))]
    sessions = [y for _,y,_ in sorted(zip(subjects,sessions,paths))]
    
    # compile data
    if net_adj:
        data = {}
        data = compileNetworkAdjacencyMatrices(paths,subjects,sessions,data)
        np.save(outPath,data)
    else:
        data = compileData(paths,subjects,sessions,data)
    
        # output data structure for records and any further analyses
        # subjects.csv
        data.to_csv(outPath,index=False)
    
    return data

### dataframe manipulations
## cut nodes for profilometry / timeseries data
def cutNodes(data,num_nodes,dataPath,foldername,savename):

    # identify inner n nodes based on num_nodes input
    total_nodes = len(data['nodeID'].unique())
    cut_nodes = int((total_nodes - num_nodes) / 2)

    # remove cut_nodes from dataframe
    data = data[data['nodeID'].between((cut_nodes)+1,(num_nodes+cut_nodes))]

    # replace empty spaces with nans
    data = data.replace(r'^\s+$', np.nan, regex=True)

    # output data structure for records and any further analyses
    if not os.path.exists(dataPath):
        os.mkdir(dataPath)

    data.to_csv(dataPath+'/'+foldername+'-'+savename+'.csv',index=False)

    return data

## will compute mean dataframe 
def computeMeanData(dataPath,data,outname):

    # make mean data frame
    data_mean =  data.groupby(['subjectID','classID','structureID']).mean().reset_index()
    data_mean['nodeID'] = [ 1 for f in range(len(data_mean['nodeID'])) ]

    # output data structure for records and any further analyses
    if not os.path.exists(dataPath):
        os.mkdir(dataPath)

    data_mean.to_csv(dataPath+outname+'.csv',index=False)

    return data_mean

### scripts related to outlier detection and reference dataframe generation
## this function will compute distance measures from input data and reference data
def computeDistance(data,references_data,measures,metric):

    # imports important distance functions
    from sklearn.metrics.pairwise import euclidean_distances
    from scipy.stats import wasserstein_distance

    # if distance metric desired is euclidean distance (i.e. for profiles), computes distance of profile from reference profile. else, just computes difference using emd
    if metric == 'euclidean':
        dist = data.groupby('subjectID',sort=False).apply(lambda x: euclidean_distances([x[measures].values.tolist(),references_data[measures].values.tolist()])[0][1]).values
    else:
        dist = data.groupby('subjectID',sort=False).apply(lambda x: wasserstein_distance(x[measures],[references_data[measures].values[0]]))

    return dist

## this function will compute simple average references for a given input data
def computeReferences(x,groupby_measures,index_measure,diff_measures):
    
    # computes mean and sd of the measures in a dataframe
    references_mean = x.groupby(groupby_measures).mean().reset_index(index_measure)
    references_sd = x.groupby(groupby_measures).std().reset_index(index_measure)
    references_sd[diff_measures] = references_sd[diff_measures] * 2
    
    return references_mean, references_sd

## this function calls computeReferences and computeDistances to create a dataframe of distance measures
def createDistanceDataframe(data,structures,groupby_measure,measures,dist_metric):
    
    # set up output lists that we will append to
    dist = []
    subj = []
    meas = []
    struc = []

    # loop through appropriate structures
    for i in structures:
        print(i)
        # set data for a given structure
        subj_data = data.loc[data['structureID'] == i]
        # compute reference for given structure
        references_data = computeReferences(subj_data,groupby_measure,groupby_measure,measures)
        # loop through measures and compute distance from reference
        for m in measures:
            if dist_metric == 'euclidean':
                dist = np.append(dist,computeDistance(subj_data,references_data[0],m,'euclidean'))
            else:
                dist = np.append(dist,computeDistance(subj_data,references_data[0],m,'emd'))
            
            # append data to appropriate lists
            subj = np.append(subj,subj_data.subjectID.unique().tolist())
            meas = np.append(meas,[ m for f in range(len(subj_data.subjectID.unique().tolist())) ])
            struc = np.append(struc,[ i for f in range(len(subj_data.subjectID.unique().tolist())) ])

    # create distance dataframe
    dist_dataframe = pd.DataFrame()
    dist_dataframe['subjectID'] = subj
    dist_dataframe['structureID'] = struc
    dist_dataframe['measures'] = meas
    dist_dataframe['distance'] = dist
    
    return dist_dataframe

## this function is useful for saving reference.jsons for a given structure
def outputReferenceJson(ref_data,measures,profile,resample_points,sourceID,data_dir,filename):
    
    # load module for resampling profile data for generating reference.json
    from scipy.signal import resample

    # loop through structures in dataframe
    for st in ref_data.structureID.unique():
        # set up important measures
        reference_json = []
        tmp = {}
        tmp['structurename'] = st
        tmp['source'] = sourceID
        # loop through measures
        for meas in measures:
            # grab data
            tmp[meas] = {}
            if profile:
                gb_frame = ref_data.loc[ref_data['structureID'] == st][['nodeID',meas]].dropna().groupby('nodeID')[meas]
            else:
                gb_frame = ref_data.loc[ref_data['structureID'] == st][[meas]].dropna()[meas]
            
            # if resample_points is a value, resamples references and computes multiple summary measures.
            # else, will just output the entire data. this second option is really only useful for non-profile/series data
            data_tmp = []
            if resample_points:                
                mean_tmp = resample(gb_frame.mean().values.tolist(),resample_points).tolist()
                min_tmp = resample(gb_frame.min().values.tolist(),resample_points).tolist()
                max_tmp = resample(gb_frame.max().values.tolist(),resample_points).tolist()
                sd_tmp = resample(gb_frame.std().values.tolist(),resample_points).tolist()
                five_tmp = resample(gb_frame.quantile(q=.05).values.tolist(),resample_points).tolist()
                twofive_tmp = resample(gb_frame.quantile(q=.25).values.tolist(),resample_points).tolist()
                sevenfive_tmp = resample(gb_frame.quantile(q=.75).values.tolist(),resample_points).tolist()
                ninefive_tmp = resample(gb_frame.quantile(q=.95).values.tolist(),resample_points).tolist()
                tmp[meas]['mean'] = mean_tmp
                tmp[meas]['min'] = min_tmp
                tmp[meas]['max'] = max_tmp
                tmp[meas]['sd'] = sd_tmp
                tmp[meas]['5_percentile'] = five_tmp
                tmp[meas]['25_percentile'] = twofive_tmp
                tmp[meas]['75_percentile'] = sevenfive_tmp
                tmp[meas]['95_percentile'] = ninefive_tmp
            else:
                data_tmp = gb_frame.values.tolist()
                tmp[meas]['data'] = data_tmp
        reference_json.append(tmp)

        with open(data_dir+'/'+filename+'_'+st+'.json','w') as ref_out_f:
            json.dump(reference_json,ref_out_f)
    
    return reference_json

## this function is used to build the reference dataset removing any subjects identified as outliers. the dataframe may or may not be useful
def buildReferenceData(data,outliers,profile,data_dir,filename):
    
    # set up dataframe
    reference_data = pd.DataFrame()

    # loop through structures and measures and set data
    for s in outliers.structureID.unique():
        for m in outliers.measures.unique():
            if profile:
                meas = ['structureID','subjectID','nodeID',m]
            else:
                meas = ['structureID','subjectID',m]
            tmpdata = data[(data["structureID"] == s) & (~data['subjectID'].isin(outliers.loc[outliers['structureID'] == s].loc[outliers['measures'] == m].subjectID.unique()))][meas].reset_index(drop=True)
            reference_data = pd.concat([reference_data,tmpdata])
    # if not profile, will compute average
    if not profile:
        reference_data = reference_data.groupby(['structureID','subjectID']).mean().reset_index()

    reference_data.to_csv(data_dir+'/'+filename+'.csv',index=False)

    return reference_data

## this function will identify if a given subjects' data is an outlier based on distance from reference and a threshold percentage
def computeOutliers(distances,threshold):
    
    # set up dataframe
    outliers = pd.DataFrame()
    
    # loop through structureas and measures and identify outliers based on thereshold distance
    for i in distances.structureID.unique():
        for m in distances.measures.unique():
            tmpdata = distances.loc[distances['structureID'] == i].loc[distances['measures'] == m]
            outliers = pd.concat([outliers,tmpdata[tmpdata['distance'] > np.percentile(tmpdata['distance'],threshold)]])
            
    return outliers

## this function calls computeOutliers, createDistanceDataframe, and buildReferenceData and outputReferenceJson to actually generate the outliers
## and final reference datasets
def outlierDetection(data,structures,groupby_measure,measures,threshold,dist_metric,build_outliers,profile,resample_points,sourceID,data_dir,filename):
    
    # set up important lists
    outliers_subjects = []
    outliers_structures = []
    outliers_measures = []
    outliers_metrics = []

    # compute distances and identify outliers
    distances = createDistanceDataframe(data,structures,groupby_measure,measures,dist_metric)
    outliers_dataframe = computeOutliers(distances,threshold)
    
    # if building references, build the reference data. otherwise, output a blank array
    if build_outliers:
        reference_dataframe = buildReferenceData(data,outliers_dataframe,profile,data_dir,filename)
        reference_json = outputReferenceJson(reference_dataframe,measures,profile,resample_points,sourceID,data_dir,filename)
    else:
        reference_dataframe = []
        reference_json = []
        
    return distances, outliers_dataframe, reference_dataframe, reference_json

## this function is useful in identifying subjects who may have had a flipped profile as compared to a reference profile.
## calls computeReferences, computeDistance.
## logic: if the distance of a subject's tract profile from a reference profile is positive and greater than a threshold percentage, then it's likely
## the data has been flipped. needs more work for all use cases. works well for easy examples like uncinate
def profileFlipCheck(data,subjects,structures,test_measure,flip_measures,dist_metric,threshold,outPath):
    
    # set up important lists
    flipped_subjects = []
    flipped_structures = []
    distance = []
    flipped_distance = []

    # loop through structures
    for i in structures:
        print(i)
        # set data frame for structure, including a copy that has the test_measure flipped
        struc_data = data.loc[data['structureID'] == i]
        flipped_struc_data = struc_data.copy()
        flipped_struc_data[test_measure] = flipped_struc_data.groupby('subjectID',sort=False).apply(lambda x: np.flip(x['fa'])).tolist()

        # build reference data
        references_data = computeReferences(struc_data,'nodeID','nodeID',flip_measures)

        # compute distances for normal data and for flipped. then compute difference
        dist = computeDistance(struc_data,references_data[0],test_measure,dist_metric)
        dist_flipped = computeDistance(flipped_struc_data,references_data[0],test_measure,dist_metric)
        differences = dist - dist_flipped

        # identify threshold of distances based on percentile. identify those that have positive differences and are greater than the threshold.
        # if so, append information
        percentile_threshold = np.percentile(differences,threshold)
        for m in range(len(differences)):
            if differences[m] > 0 and differences[m] > percentile_threshold:
                flipped_subjects = np.append(flipped_subjects,subjects[m])
                flipped_structures = np.append(flipped_structures,i)
                distance = np.append(distance,dist[m])
                flipped_distance = np.append(flipped_distance,dist_flipped[m])

    # generate ouput dataframe containing flipped subject data
    output_summary = pd.DataFrame()
    output_summary['flipped_subjects'] = flipped_subjects
    output_summary['flipped_structures'] = flipped_structures
    output_summary['distance'] = distance
    output_summary['flipped_distance'] = flipped_distance

    if outPath:
        output_summary.to_csv(outPath+'_flipped_profiles.csv',index=False)
        
# this function will merge the structural and diffusion data for the reference datasets
def mergeStructuralDiffusionJson(data,structuralPath,diffusionPath,outPath):
    for i in data.structureID.unique():
        print(i)
        with open(structuralPath+'_'+i+'.json','r') as structural_f:
            structural = json.load(structural_f)

        with open(diffusionPath+'_'+i+'.json','r') as diffusion_f:
            diffusion = json.load(diffusion_f)

        merged = {**structural[0],**diffusion[0]}

        with open(outPath+'_'+i+'.json','w') as out_f:
            json.dump(merged,out_f)

### adjacency-matrix related fuctions for computing network values locally
def binarizeMatrices(data):
    
    # use brain connectivity toolbox to binarize data
    bin_data = [ bct.utils.binarize(data[f]) for f in data.keys() ]
    
    return bin_data

def thresholdMatrices(data,bin_data,thresholdPercentageSubjects):
    
    # compute sum of matrices
    sum_mat = np.zeros(np.shape(bin_data)[1:])
    
    for i in bin_data:
        sum_mat = sum_mat + i
        
    # compute threshold value
    thrs = thresholdPercentageSubjects*len(bin_data)
    
    # loop through data and make those nodes that dont meet threshold 0
    for i in data.keys():
        data[i][sum_mat < thrs] = 0
        
    return data

def computeMeanNetworkConnectivity(data,networks,indices,out_path):

    mean_data = []
    subs = []
    nets = []

    for l in data.keys():
        for n in networks:
            tmpdata = []
            subs = np.append(subs,l)
            nets = np.append(nets,n)
            for i in indices[n]:
                for j in indices[n]:
                    if i != j:
                        tmpdata = np.append(tmpdata,data[l][int(i)][int(j)])
            mean_data = np.append(mean_data,np.mean(tmpdata))

    out_df = pd.DataFrame()
    out_df['subjectID'] = [ f.split('_sess1')[0] for f in subs ]
    out_df['FC'] = mean_data
    out_df = pd.merge(out_df,subjects_data,on='subjectID')
    out_df['structureID'] = nets

    if out_path:
        out_df.to_csv(out_path,index=False)
        
    return out_df

### scatter plot related scripts
# groups data by input measure and computes mean for each value in that column. x_stat is a pd dataframe, with each row being a single value, and each column being a different ID value or measure
def averageWithinColumn(x_stat,y_stat,x_measure,y_measure,measure):

    X = x_stat.groupby(measure).mean()[x_measure].tolist()
    Y = y_stat.groupby(measure).mean()[y_measure].tolist()

    return X,Y

# groups data by input measure and creates an array by appending data into x and y arrays. x_stat and y_stat are pd dataframes, with each row being a single value, and each column being a different ID value or measure
# designed for test retest. x_stat and y_stat should have the same number of rows. but more importantly, should correspond to the same source (i.e. subject)
# can be same pd.dataframe, but indexing of specific subject groups
def appendWithinColumn(x_stat,y_stat,x_measure,y_measure,measure):

    X,Y = [np.array([]),np.array([])]
    for i in range(len(x_stat[measure].unique())):
        x = x_stat[x_stat[measure] == x_stat[measure].unique()[i]][x_measure]
        y = y_stat[y_stat[measure] == y_stat[measure].unique()[i]][y_measure]

        if np.isnan(x).any() or np.isnan(y).any():
            print("skipping %s due to nan" %x_stat[measure].unique()[i])
        else:
            # checks to make sure the same data
            if len(x) == len(y):
                X = np.append(X,x)
                Y = np.append(Y,y)

    return X,Y

# unravels networks. x_stat and y_stat should be S x M, where S is the number of subjects and M is the adjacency matrix for that subject
def ravelNetwork(x_stat,y_stat):

    import numpy as np

    X = np.ravel(x_stat).tolist()
    Y = np.ravel(y_stat).tolist()

    return X,Y

# unravels nonnetwork data. x_stat and y_stat should be pd dataframes. x_measure and y_measure are the measure to unrvavel. 
# designed for test retest. x_stat and y_stat should have the same number of rows. but more importantly, should correspond to the same source (i.e. subject)
# can be same pd.dataframe, but indexing of specific subject groups
def ravelNonNetwork(x_stat,y_stat,x_measure,y_measure):

    X = x_stat[x_measure].to_list()
    Y = y_stat[y_measure].to_list()

    return X,Y

# wrapper function to call either of the above scripts based on user input
def setupData(x_data,y_data,x_measure,y_measure,ravelAverageAppend,isnetwork,measure):

    x_stat = x_data
    y_stat = y_data

    if ravelAverageAppend == 'average':
        X,Y = averageWithinColumn(x_stat,y_stat,x_measure,y_measure,measure)
    elif ravelAverageAppend == 'append':
        X,Y = appendWithinColumn(x_stat,y_stat,x_measure,y_measure,measure)
    elif ravelAverageAppend == 'ravel':
        if isnetwork == True:
            X,Y = ravelNetwork(x_stat,y_stat)
        else:
            X,Y = ravelNonNetwork(x_stat,y_stat,x_measure,y_measure)

    return x_stat,y_stat,X,Y

# function to shuffle data and colors
def shuffleDataAlg(X,Y,hues):

    from sklearn.utils import shuffle

    X,Y,hues = shuffle(X,Y,hues)

    return X,Y,hues

# simple display or figure save function
def saveOrShowImg(dir_out,x_measure,y_measure,img_name):
    import os,sys 
    import matplotlib.pyplot as plt
    import warnings
    
    with warnings.catch_warnings():
        # this will suppress all warnings in this block
        warnings.simplefilter("ignore")
 
        # save or show plot
        if dir_out:
            if not os.path.exists(dir_out):
                os.mkdir(dir_out)

            if x_measure == y_measure:
                img_name_eps = img_name+'_'+x_measure+'.eps'
                img_name_png = img_name+'_'+x_measure+'.png'
                img_name_svg = img_name+'_'+x_measure+'.svg'
            else:
                img_name_eps = img_name+'_'+x_measure+'_vs_'+y_measure+'.eps'
                img_name_png = img_name+'_'+x_measure+'_vs_'+y_measure+'.png'
                img_name_svg = img_name+'_'+x_measure+'_vs_'+y_measure+'.svg'

            plt.savefig(os.path.join(dir_out, img_name_eps),transparent=True)
            plt.savefig(os.path.join(dir_out, img_name_png))     
    #         plt.savefig(os.path.join(dir_out, img_name_svg))
        else:
            plt.show()

        plt.close()
    
# uses seaborn's relplot function to plot data for each unique value in a column of a pandas dataframe (ex. subjects, structureID). useful for supplementary figures or sanity checking or preliminary results
# column measure is the measure within which each unique value will have its own plot. hue_measure is the column to use for coloring the data. column_wrap is how many panels you want per row
# trendline, depending on user input, can either be the linear regression between x_data[x_measure] and y_data[y_measure] or the line of equality
# dir_out and img_name are the directory where the figures should be saved and the name for the image. will save .eps and .png
# if want to view plot instead of save, set dir_out=""
def relplotScatter(x_data,y_data,x_measure,y_measure,column_measure,hue_measure,column_wrap,trendline,dir_out,img_name):

    import os,sys
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.metrics import mean_squared_error

    # grab data: CANNOT BE AVERAGE
    [x_stat,y_stat,X,Y] = setupData(x_data,y_data,x_measure,y_measure,'ravel',False,hue_measure)

    p = sns.relplot(x=X,y=Y,col=x_stat[column_measure],hue=x_stat[hue_measure],kind="scatter",s=100,col_wrap=column_wrap)

    # setting counter. looping through axes to add important info and regression lines
    i = 0
    for ax in p.axes.flat:
        x_lim,y_lim = [ax.get_xlim(),ax.get_ylim()]

        if trendline == 'equality':
            ax.plot(x_lim,y_lim,ls="--",c='k')
        elif trendline == 'linreg':
            m,b = np.polyfit(p.data[p.data[column_measure] == x_stat[column_measure].unique()[i]]['x'],p.data[p.data[column_measure] == y_stat[column_measure].unique()[i]]['y'],1)
            ax.plot(ax.get_xticks(),m*ax.get_xticks() + b)
            plt.text(0.1,0.7,'y = %s x + %s' %(str(np.round(m,4)),str(np.round(b,4))),fontsize=12,verticalalignment="top",horizontalalignment="left",transform=ax.transAxes)

        ax.set_xlim(x_lim)
        ax.set_ylim(y_lim)
        ax.set_xlabel(x_measure)
        ax.set_ylabel(y_measure)

        # compute correlation for each subject and add to plots
        corr = np.corrcoef(p.data[p.data[column_measure] == x_stat[column_measure].unique()[i]]['x'],p.data[p.data[column_measure] == y_stat[column_measure].unique()[i]]['y'])[1][0]
        plt.text(0.1,0.9,'r = %s' %str(np.round(corr,4)),fontsize=12,verticalalignment="top",horizontalalignment="left",transform=ax.transAxes)

        # compute rmse for each subject and add to plots
        rmse = np.sqrt(mean_squared_error(p.data[p.data[column_measure] == x_stat[column_measure].unique()[i]]['x'],p.data[p.data[column_measure] == y_stat[column_measure].unique()[i]]['y']))
        plt.text(0.1,0.8,'rmse = %s' %str(np.round(rmse,4)),fontsize=12,verticalalignment="top",horizontalalignment="left",transform=ax.transAxes)

        # update counter
        i = i+1

    # save image or show image
    saveOrShowImg(dir_out,x_measure,y_measure,img_name)

# uses seaborn's scatter function to plot data from x_data[x_measure] and y_data[y_measure]. useful for publication worthy figure
# column measure is the measure within which data will be summarized. hue_measure is the column to use for coloring the data. 
# ravelAverageAppend is a string value of either 'append' to use the append function, 'ravel' to use the ravel function, or 'average' to use the average function
# trendline, depending on user input, can either be the linear regression between x_data[x_measure] and y_data[y_measure] or the line of equality
# dir_out and img_name are the directory where the figures should be saved and the name for the image. will save .eps and .png
# if want to view plot instead of save, set dir_out=""
def singleplotScatter(colors_dict,x_data,y_data,x_measure,y_measure,logX,column_measure,hue_measure,ravelAverageAppend,trendline,shuffleData,colorDistance,dir_out,img_name):

    import os,sys
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.metrics import mean_squared_error

    # grab data
    [x_stat,y_stat,X,Y] = setupData(x_data,y_data,x_measure,y_measure,ravelAverageAppend,False,column_measure)
    
    if colorDistance:
        category = colorDistanceScatter(X,Y)
    else:
        colors = sns.color_palette('colorblind',len(x_stat[hue_measure]))

    if ravelAverageAppend == 'average':
        if isinstance(x_stat[hue_measure].unique()[0],str):
            hues = x_stat[hue_measure].unique().tolist()
        else:
            hues = x_stat.groupby(column_measure).mean()[hue_measure].tolist()
    else:
        hues = list(x_stat[hue_measure])

    if shuffleData == True:
        X,Y,hues = shuffleDataAlg(X,Y,hues)

    if logX == True:
        X = np.log10(X)

    if colors_dict:
        p = sns.scatterplot(x=X,y=Y,hue=hues,s=100,palette=colors_dict,legend=False)
    elif colorDistance:
        p = sns.scatterplot(x=X,y=Y,hue=category,hue_order=['two-sd','one-sd','lt-one-sd'],palette=['red','green','blue'],s=100)
    else:
        p = sns.scatterplot(x=X,y=Y,hue=hues,s=100)

    # set x and ylimits, plot line of equality, and legend
    if x_measure == y_measure:
        p.axes.axis('square')
        y_ticks = p.axes.get_yticks()
        p.axes.set_xticks(y_ticks)
        p.axes.set_yticks(p.axes.get_xticks())
        p.axes.set_ylim(p.axes.get_xlim())
        p.axes.set_xlim(p.axes.get_xlim())


    x_lim,y_lim = [p.axes.get_xlim(),p.axes.get_ylim()]

    # trendline: either equality or linear regression
    if trendline == 'equality':
        p.plot(x_lim,y_lim,ls="--",c='k')
        ax = plt.gca()
        ax.get_legend().remove()
    elif trendline == 'linreg':
        m,b = np.polyfit(X,Y,1)
        p.plot(p.get_xticks(),m*p.get_xticks() + b,c='k')
        plt.text(0.1,0.7,'y = %s x + %s' %(str(np.round(m,4)),str(np.round(b,4))),fontsize=16,verticalalignment="top",horizontalalignment="left",transform=p.axes.transAxes)
        ax = plt.gca()
        ax.get_legend().remove()

    elif trendline == 'groupreg':
        for g in range(len(groups)):
            if stat_name == 'volume':
                slope, intercept, r_value, p_value, std_err = stats.linregress(np.log10(stat[['structureID',stat_name]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[stat_name]),stat[['structureID',dm]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[dm])
                ax = sns.regplot(x=np.log10(stat[['structureID',stat_name]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[stat_name]),y=stat[['structureID',dm]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[dm],color=colors[groups[g]],scatter=True,line_kws={'label':"y={0:.5f}x+{1:.4f}".format(slope,intercept)})

            else:
                slope, intercept, r_value, p_value, std_err = stats.linregress(stat[['structureID',stat_name]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[stat_name],stat[['structureID',dm]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[dm])
                ax = sns.regplot(x=stat[['structureID',stat_name]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[stat_name],y=stat[['structureID',dm]][stat['subjectID'].str.contains('%s_' %str(g+1))].groupby('structureID',as_index=False).mean()[dm],color=colors[groups[g]],scatter=True,line_kws={'label':"y={0:.5f}x+{1:.4f}".format(slope,intercept)})

            ax.legend()

    # compute correlation for each subject and add to plots
    corr = np.corrcoef(X,Y)[1][0]
    plt.text(0.1,0.9,'r = %s' %str(np.round(corr,4)),fontsize=16,verticalalignment="top",horizontalalignment="left",transform=p.axes.transAxes)

    # compute rmse for each subject and add to plots
    rmse = np.sqrt(mean_squared_error(X,Y))
    plt.text(0.1,0.8,'rmse = %s' %str(np.round(rmse,4)),fontsize=16,verticalalignment="top",horizontalalignment="left",transform=p.axes.transAxes)

    # set title and x and y labels
    plt.title('%s vs %s' %(x_measure,y_measure),fontsize=20)
    plt.xlabel(x_measure,fontsize=18)
    plt.ylabel(y_measure,fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    # remove top and right spines from plot
    p.axes.spines["top"].set_visible(False)
    p.axes.spines["right"].set_visible(False)

    # save image or show image
    saveOrShowImg(dir_out,x_measure,y_measure,img_name)
    
def colorDistanceScatter(x,y):
    
    # set output variable (category) and compute ratio and difference from 1 (i.e. perfect ratio)
    category = []
    ratio = [ x[i] / y[i] if y[i] > 0 else 5 for i in range(len(x)) ]
    diff = [ np.abs(ratio[i] - 1) for i in range(len(ratio)) ]
    
    # compute standard deviation thresholds for difference values
    one_sd = np.std(diff)
    two_sd = one_sd * 2
    
    # loop through each data point and determine category (one sd: within 1 and 2 sds, two-sd: greater or equal to 2 sds, lt-one-sd: within one sd)
    for i in diff:
        if i >= two_sd:
            category = np.append(category,'two-sd')
        elif one_sd <= i < two_sd:
            category = np.append(category,'one-sd')
        else:
            category = np.append(category,'lt-one-sd')
    
    return category

In [4]:
### setting up variables and adding paths
print("setting up variables")
topPath = "./"
os.chdir(topPath)
data_dir = topPath+'/data/'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
img_dir = topPath+'/img/'
if not os.path.exists(img_dir):
    os.mkdir(img_dir)

groups = ['hcp']
colors_array = ['blue']
diff_micro_measures = ['ad','fa','md','rd','ndi','odi','isovf']
print("setting up variables complete")

### grabbing subjects demographic data
print("grabbing demographic data")
subjects_data = pd.read_csv('./subjects_data.csv')
print("grabbing demographic data complete")

if subjects_data['subjectID'].dtype == 'int':
    subjects_data['subjectID'] = [ str(int(np.float(f))) for f in subjects_data.subjectID ]
    
subjects_data['classID'] = [ 'hcp' for f in subjects_data['subjectID'] ]
subjects_data['colors'] = [ 'blue' for f in subjects_data['subjectID'] ]

colors = {}
subjects = {}

# loop through groups and identify subjects and set color schema for each group
for g in range(len(groups)):
    # set subjects array
    subjects[groups[g]] =  subjects_data['subjectID']
    subjects[groups[g]].sort_values()
    
    # set colors array
    colors_name = colors_array[g]
    colors[groups[g]] = colors_array[g]

# create subjects color dictionary
colors_dict = createColorDictionary(subjects_data,'subjectID','colorblind')

setting up variables
setting up variables complete
grabbing demographic data
grabbing demographic data complete


In [ ]:
### structural and diffusion data in cortical and subcortical structures
print("computing gray matter parcel analyses")

# create data structures
# cortical measures - glasser
if os.path.isfile(data_dir+'/cortical_glasser.csv'):
    cortical_glasser = pd.read_csv(data_dir+'/cortical_glasser.csv')
else:
    cortical_glasser =  collectData('neuro/parc-stats',['cortex_mapping_stats'],['glasser'],'parc_MEAN.csv',data_dir+'/cortical_glasser.csv')
    cortical_glasser = cortical_glasser.drop_duplicates()
    cortical_glasser['subjectID'] = cortical_glasser.subjectID.astype(str)
    cortical_glasser = pd.merge(cortical_glasser,subjects_data,on='subjectID')
    cortical_glasser.to_csv(data_dir+'/cortical_glasser.csv',index=False)

# replace vertex volume measure with overall parcel volume
if os.path.isfile(data_dir+'/cortical_glasser_parcel_volume.csv'):
    cortical_glasser_parcel_volume = pd.read_csv(data_dir+'/cortical_glasser_parcel_volume.csv')
else:
    cortical_glasser_parcel_volume = collectData('neuro/parc-stats',['surface'],['glasser'],'cortex.csv',data_dir+'/cortical_glasser_parcel_volume.csv')
    cortical_glasser_parcel_volume = cortical_glasser_parcel_volume.drop_duplicates()
    cortical_glasser_parcel_volume['subjectID'] = cortical_glasser_parcel_volume.subjectID.astype(str)
    cortical_glasser_parcel_volume = cortical_glasser_parcel_volume[~cortical_glasser_parcel_volume['structureID'].isin(["lh_unknown_0","rh_unknown_0","lh_???","rh_???"])]
    cortical_glasser_parcel_volume['structureID'] = [ f.split('h_')[1] for f in cortical_glasser_parcel_volume['structureID'] ]
    cortical_glasser_parcel_volume.to_csv(data_dir+'/cortical_glasser_parcel_volume.csv',index=False)

cortical_glasser.drop(columns=['volume','thickness'],inplace=True)
cortical_glasser = pd.merge(cortical_glasser,cortical_glasser_parcel_volume,on=['subjectID','structureID','nodeID','sessionID'])

# cortical measures -  aparc.a2009s
if os.path.isfile(data_dir+'/cortical_aparc.csv'):
    cortical_aparc = pd.read_csv(data_dir+'/cortical_aparc.csv')
else:
    cortical_aparc =  collectData('neuro/parc-stats',['cortex_mapping_stats'],['glasser'],'aparc_MEAN.csv',data_dir+'/cortical_aparc.csv')
    cortical_aparc = cortical_aparc.drop_duplicates()
    cortical_aparc['subjectID'] = cortical_aparc.subjectID.astype(str)
    cortical_aparc = pd.merge(cortical_aparc,subjects_data,on='subjectID')
    cortical_aparc.to_csv(data_dir+'/cortical_aparc.csv',index=False)

# freesurfer measures: aparc
if os.path.isfile(data_dir+'/cortical_aparc_parcel_volume.csv'):
    cortical_aparc_parcel_volume = pd.read_csv(data_dir+'/cortical_aparc_parcel_volume.csv')
else:
    cortical_aparc_parcel_volume = collectData('neuro/parc-stats',['freesurfer'],['aparc.a2009s'],'cortex.csv',data_dir+'/cortical_aparc_parcel_volume.csv')
    cortical_aparc_parcel_volume = cortical_aparc_parcel_volume.drop_duplicates()
    cortical_aparc_parcel_volume['subjectID'] = cortical_aparc_parcel_volume.subjectID.astype(str)
    cortical_aparc_parcel_volume.to_csv(data_dir+'/cortical_aparc_parcel_volume.csv',index=False)

cortical_aparc.drop(columns=['volume','thickness'],inplace=True)
cortical_aparc = pd.merge(cortical_aparc,cortical_aparc_parcel_volume,on=['subjectID','structureID','nodeID','sessionID'])

# concatenate glasser and aparc
if os.path.isfile(data_dir+'/cortical.csv'):
    cortical = pd.read_csv(data_dir+'/cortical.csv')
else:
    cortical_glasser['parcellation'] = [ 'glasser' for f in cortical_glasser['subjectID']]
    cortical_aparc['parcellation'] = [ 'aparc.a2009s' for f in cortical_aparc['subjectID']]
    cortical = pd.concat([cortical_glasser,cortical_aparc])
    cortical.to_csv(data_dir+'/cortical.csv',index=False)

# # # subcortical measures
if os.path.isfile(data_dir+'/subcortical.csv'):
    subcortical = pd.read_csv(data_dir+'/subcortical.csv')
else:
    subcortical =  collectData('neuro/parc-stats',['subcort_stats'],['tensor'],'aseg_nodes.csv',data_dir+'/subcortical.csv')
    subcortical = subcortical.drop_duplicates()
    subcortical['subjectID'] = subcortical.subjectID.astype(str)
    subcortical = pd.merge(subcortical,subjects_data,on='subjectID')
    subcortical.to_csv('subcortical.csv',index=False)

# # clean and remove data that is outside physical bounds: cortical
if os.path.isfile(data_dir+'/cortical_cleaned.csv'):
    cortical = pd.read_csv(data_dir+'/cortical_cleaned.csv')
else:
    bad_cortical = pd.DataFrame()
    bad_cortical = pd.concat([bad_cortical,cortical.loc[cortical['ad'] > 3]])
    bad_cortical = pd.concat([bad_cortical,cortical.loc[cortical['ad'] < 0]])
    bad_cortical = pd.concat([bad_cortical,cortical.loc[cortical['fa'] > 1]])
    bad_cortical = pd.concat([bad_cortical,cortical.loc[cortical['fa'] < 0]])
    bad_cortical = pd.concat([bad_cortical,cortical.loc[cortical['md'] > 3]])
    bad_cortical = pd.concat([bad_cortical,cortical.loc[cortical['md'] < 0]])
    bad_cortical = pd.concat([bad_cortical,cortical.loc[cortical['rd'] > 3]])
    bad_cortical = pd.concat([bad_cortical,cortical.loc[cortical['rd'] < 0]])
    cortical = cortical.loc[~cortical.index.isin(bad_cortical.index)]
    cortical.to_csv(data_dir+'/cortical_cleaned.csv',index=False)

# # clean and remove data that is outside physical bounds: subcortical
if os.path.isfile(data_dir+'/subcortical_cleaned.csv'):
    subcortical = pd.read_csv(data_dir+'/subcortical_cleaned.csv')
else:
    bad_subcortical = pd.DataFrame()
    bad_subcortical = pd.concat([bad_subcortical,subcortical.loc[subcortical['ad'] > 3]])
    bad_subcortical = pd.concat([bad_subcortical,subcortical.loc[subcortical['ad'] < 0]])
    bad_subcortical = pd.concat([bad_subcortical,subcortical.loc[subcortical['fa'] > 1]])
    bad_subcortical = pd.concat([bad_subcortical,subcortical.loc[subcortical['fa'] < 0]])
    bad_subcortical = pd.concat([bad_subcortical,subcortical.loc[subcortical['md'] > 3]])
    bad_subcortical = pd.concat([bad_subcortical,subcortical.loc[subcortical['md'] < 0]])
    bad_subcortical = pd.concat([bad_subcortical,subcortical.loc[subcortical['rd'] > 3]])
    bad_subcortical = pd.concat([bad_subcortical,subcortical.loc[subcortical['rd'] < 0]])
    subcortical = subcortical.loc[~subcortical.index.isin(bad_subcortical.index)]
    subcortical.to_csv(data_dir+'/subcortical_cleaned.csv',index=False)

In [ ]:
### diffusion snr
# diffusion snr data
if os.path.isfile(data_dir+'/diffusion-callosal-snr.csv'):
    diffusion_snr_data = pd.read_csv(data_dir+'/diffusion-callosal-snr.csv')
else:
    diffusion_snr_data = collectData('neuro/snr-stats',['preprocessed'],['dwi_snr'],'snr.csv',data_dir+'/diffusion-callosal-snr.csv')
    diffusion_snr_data['subjectID'] = diffusion_snr_data.subjectID.astype(str)
    diffusion_snr_data = diffusion_snr_data.drop_duplicates()
    diffusion_snr_data.rename(columns={'volumes': 'structureID'},inplace=True)
    diffusion_snr_data.to_csv(data_dir+'/diffusion-callosal-snr.csv',index=False)

In [ ]:
### structural and diffusion data in white matter tracts
print("computing tractometry analyses")

# create data structures
# tractmeasures data
if os.path.isfile(data_dir+'/tractmeasures-total_nodes.csv'):
    track_data = pd.read_csv(data_dir+'/tractmeasures-total_nodes.csv')
else:
    track_data = collectData('neuro/tractmeasures',['macro_micro'],['full_macro_stats'],'tractmeasures.csv',data_dir+'/tractmeasures-total_nodes.csv')
    track_data['subjectID'] = track_data.subjectID.astype(str)
    track_data = track_data.drop_duplicates()
    track_data = track_data.loc[track_data['structureID'] != 'wbfg' ]
    track_data.to_csv(data_dir+'/tractmeasures-total_nodes.csv',index=False)
    
# remove subjects with missing data
subjs_with_nans = track_data[track_data.isna()['fa']].subjectID.unique()

# check for missing individual tracts
subjs_missing_tracts = [ f for f in track_data.subjectID.unique().tolist() if len(track_data[track_data['subjectID'] == f].structureID.unique()) < 61 ]
subjs_missing_data = [*subjs_with_nans,*subjs_missing_tracts]

# select only subjects that don't have missing data
track_data = track_data.loc[~track_data['subjectID'].isin(subjs_missing_data)]

# # merge subjects_data
if os.path.isfile(data_dir+'/tractmeasures-total_nodes_merged.csv'):
    track_data = pd.read_csv(data_dir+'/tractmeasures-total_nodes_merged.csv')
    track_data = track_data.drop_duplicates()
else:
    track_data = pd.merge(track_data,subjects_data,on='subjectID')
    track_data.to_csv(data_dir+'/tractmeasures-total_nodes_merged.csv')

# # cut nodes
if os.path.isfile(data_dir+'/tractmeasures-cut_nodes.csv'):
    track_data_cut = pd.read_csv(data_dir+'/tractmeasures-cut_nodes.csv')
else:
    track_data_cut = cutNodes(track_data,180,data_dir,'tractmeasures','cut_nodes')
    track_data_cut = pd.merge(track_data_cut,subjects_data,on=['subjectID','classID','age'])
    track_data_cut.to_csv(data_dir+'/tractmeasures-cut_nodes.csv',index=False)

# # compute mean data
if os.path.isfile(data_dir+'/tractmeasures-mean_nodes.csv'):
    track_data_mean = pd.read_csv(data_dir+'/tractmeasures-mean_nodes.csv')
else:
    track_data_mean = computeMeanData(data_dir,track_data_cut,'tractmeasures-mean_nodes')
    track_data_mean = pd.merge(track_data_mean,subjects_data,on=['subjectID','classID'])
    track_data_mean.to_csv(data_dir+'/tractmeasures-mean_nodes.csv',index=False)


In [5]:
#### dmri glasser structural (density) networks - max node degree vs age replication
if os.path.isfile(data_dir+'/'+groups[0]+'_dmri_glasser_preclean_adjacency.npy'):
    dmri_glasser_networks_adjacency = np.load(data_dir+'/'+groups[0]+'_dmri_glasser_preclean_adjacency.npy',allow_pickle=True).item()
else:
    dmri_glasser_networks_adjacency = collectData('generic/network',['!measurements','networkmatrices','preprocessed'],['bl_paper','ga','structural','density'],'network.json.gz',data_dir+'/'+groups[0]+'_dmri_glasser_preclean_adjacency',True)

# find binarized matrices
bin_matrix = binarizeMatrices(dmri_glasser_networks_adjacency)

# clean matrices
dmri_glasser_networks_adjacency_cleaned = thresholdMatrices(dmri_glasser_networks_adjacency,bin_matrix,0.5)

# compute node degree for each subject
dmri_glasser_networks_adjacency_cleaned_nodes = pd.DataFrame()
subs = []
degree = []
nodes = []
for i in dmri_glasser_networks_adjacency_cleaned.keys():
    subs = np.append(subs,[ i.split('_')[0] for f in range(len(dmri_glasser_networks_adjacency_cleaned[i])) ])
    nodes = np.append(nodes,[ str(f) for f in range(len(dmri_glasser_networks_adjacency_cleaned[i])) ])
    degree = np.append(degree,bct.degrees_und(dmri_glasser_networks_adjacency_cleaned[i]))

dmri_glasser_networks_adjacency_cleaned_nodes['subjectID'] = subs
dmri_glasser_networks_adjacency_cleaned_nodes['node_degree'] = degree
dmri_glasser_networks_adjacency_cleaned_nodes['node'] = nodes
dmri_glasser_networks_adjacency_cleaned_nodes = pd.merge(dmri_glasser_networks_adjacency_cleaned_nodes,subjects_data.loc[subjects_data['subjectID'].isin(dmri_glasser_networks_adjacency_cleaned_nodes['subjectID'].unique())][['subjectID','age']],on='subjectID')

bins = [18, 28, 38]
dmri_glasser_networks_adjacency_cleaned_nodes['bin'] = pd.cut(dmri_glasser_networks_adjacency_cleaned_nodes['age'],bins)
dmri_glasser_networks_adjacency_cleaned_nodes = dmri_glasser_networks_adjacency_cleaned_nodes.loc[~dmri_glasser_networks_adjacency_cleaned_nodes['node'].isin(['0','181'])]

dmri_glasser_networks_adjacency_cleaned_nodes_max = dmri_glasser_networks_adjacency_cleaned_nodes.groupby('subjectID').apply(lambda x: np.max(x['node_degree'])).reset_index()
dmri_glasser_networks_adjacency_cleaned_nodes_max.rename(columns={0: "degree"},inplace=True)
dmri_glasser_networks_adjacency_cleaned_nodes_max = pd.merge(dmri_glasser_networks_adjacency_cleaned_nodes_max,dmri_glasser_networks_adjacency_cleaned_nodes[['subjectID','age','bin']],on='subjectID')
dmri_glasser_networks_adjacency_cleaned_nodes_max['classID'] = [ groups[0] for f in range(len(dmri_glasser_networks_adjacency_cleaned_nodes_max)) ]
dmri_glasser_networks_adjacency_cleaned_nodes_max = dmri_glasser_networks_adjacency_cleaned_nodes_max.drop_duplicates()
dmri_glasser_networks_adjacency_cleaned_nodes_max.to_csv(groups[0]+'-max-node-degree.csv',index=False)

sns.heatmap(np.nan_to_num(np.log10(dmri_glasser_networks_adjacency_cleaned['103818_sess1']),neginf=-7),cmap='hot',vmin=-7,vmax=0)
plt.savefig('103818_density_net.eps',transparent=True)

In [12]:
#### fmri yeo networks - max node degree vs age replication
if os.path.isfile(data_dir+'/'+groups[0]+'_fmri_yeo_preclean_adjacency.npy'):
    fmri_yeo_networks_adjacency = np.load(data_dir+'/'+groups[0]+'_fmri_yeo_preclean_adjacency.npy',allow_pickle=True).item()
else:
    fmri_yeo_networks_adjacency = collectData('generic/network',["time series"],['yeo-mni','final'],'network.json.gz',data_dir+'/'+groups[0]+'_fmri_yeo_preclean_adjacency',True)

# grab label indices
networks = ['VisCent','VisPeri','SomMotA','SomMotB','DorsAttnA','DorsAttnB','SalVentAttnA','SalVentAttnB','Limbic','ContA','ContB','ContC','DefaultA','DefaultB','DefaultC']
with open('label.json','r') as label_f:
    labels = json.load(label_f)

indices = {}
for n in range(len(networks)):
    indices[networks[n]] = []
    for i in range(len(labels)):
        netname = labels[i]['name']
        if networks[n] in netname:
                indices[networks[n]] = np.append(indices[networks[n]],int(i))

# create average matrix
if os.path.isfile(data_dir+'/'+groups[0]+'_fmri_yeo_preclean_adjacency_mean.npy'):
    fmri_yeo_networks_adjacency_mean = np.load(data_dir+'/'+groups[0]+'_fmri_yeo_preclean_adjacency_mean.npy')
else:
    fmri_yeo_networks_adjacency_mean = fmri_yeo_networks_adjacency[list(fmri_yeo_networks.keys())[0]]

    for i in fmri_yeo_networks_adjacency.keys():
        fmri_yeo_networks_adjacency_mean = (fmri_yeo_networks_adjacency_mean + fmri_yeo_networks_adjacency[i]) / 2
    np.save(data_dir+'/'+groups[0]+'_fmri_yeo_preclean_adjacency_mean.npy',fmri_yeo_networks_adjacency_mean)
    
# compute average functional connectivity (within network) for each network in yeo17
fmri_yeo_networks_adjacency_mean_fc = computeMeanNetworkConnectivity(fmri_yeo_networks_adjacency,networks,indices,"")

In [17]:
### creating reference datasets
print("creating references for subcortical cortical and tractmeasures data")

if ~os.path.isdir(data_dir+'/references'):
    os.mkdir(data_dir+'/references')

# compute reference dataset for subcortical structures
subcortical_structural_measures = ['number_of_voxels','volume']
subcortical_structural_distances, subcortical_structural_outliers, subcortical_structural_reference_dataframe, subcortical_structural_reference_json = outlierDetection(subcortical,subcortical.structureID.unique(),'structureID',subcortical_structural_measures,95,'emd',True,False,'',groups[0],data_dir+'/references','subcortical_structural_reference_'+groups[0])
subcortical_diffusion_measures = ['ad','fa','md','rd','ndi','odi','isovf']
subcortical_diffusion_distances, subcortical_diffusion_outliers, subcortical_diffusion_reference_dataframe, subcortical_diffusion_reference_json = outlierDetection(subcortical,subcortical.structureID.unique(),'structureID',subcortical_diffusion_measures,95,'emd',True,False,'',groups[0],data_dir+'/references','subcortical_diffusion_reference_'+groups[0])

# compute reference dataset for cortical structures
cortical_diffusion_measures = ['ad','fa','md','rd','ndi','isovf','odi']    
cortical_diffusion_distances, cortical_diffusion_outliers, cortical_diffusion_reference_dataframe, cortical_diffusion_reference_json = outlierDetection(cortical,cortical.structureID.unique(),'structureID',cortical_diffusion_measures,95,'emd',True,False,"",groups[0],data_dir+'/references','cortical_diffusion_reference_'+groups[0])
cortical_structural_measures = ['number_of_vertices','surface_area_mm^2','gray_matter_volume_mm^3','average_thickness_mm','thickness_stddev_mm','integrated_rectified_mean_curvature_mm^-1','integrated_rectified_gaussian_curvature_mm^-2','folding_index','intrinsic_curvature_index']
cortical_structural_distances, cortical_structural_outliers, cortical_structural_reference_dataframe, cortical_structural_reference_json = outlierDetection(cortical,cortical.structureID.unique(),'structureID',cortical_structural_measures,95,'emd',True,False,"",groups[0],data_dir+'/references','cortical_structural_reference_'+groups[0])

# compute reference dataset for tractmeasures  
track_structural_measures = [ f for f in track_data.keys() if f not in diff_micro_measures + subjects_data.keys().tolist() + ['structureID','sessionID','nodeID','ExponentialFitA','ExponentialFitB'] + track_data.columns[track_data.isna().any()].tolist() ]
track_structural_distances, track_structural_outliers, track_structural_reference_dataframe, track_structural_reference_json = outlierDetection(track_data_mean,track_data_mean.structureID.unique(),'structureID',track_structural_measures,95,'emd',True,False,"",groups[0],data_dir+'/references','tractmeasures_structural_reference_'+groups[0])
track_diffusion_measures = ['ad','fa','md','rd']
track_diffusion_distances, track_diffusion_outliers, track_diffusion_reference_dataframe, track_diffusion_reference_json = outlierDetection(track_data,track_data.structureID.unique(),'nodeID',track_diffusion_measures,95,'euclidean',True,True,50,groups[0],data_dir+'/references','tractmeasures_diffusion_reference_'+groups[0]+'_tensor')
track_diffusion_measures = ['ndi','odi','isovf']
track_diffusion_distances, track_diffusion_outliers, track_diffusion_reference_dataframe, track_diffusion_reference_json = outlierDetection(track_data,track_data.structureID.unique(),'nodeID',track_diffusion_measures,95,'euclidean',True,True,50,groups[0],data_dir+'/references','tractmeasures_diffusion_reference_'+groups[0]+'_noddi')

## compute reference dataset of fmri within network FC
fmri_within_network_fc_measures = ['FC']
fmri_within_network_fc_distances, fmri_within_network_fc_outliers, fmri_within_network_fc_reference_dataframe, fmri_within_network_fc_reference_json = outlierDetection(fmri_within_network_fc,subcortical.structureID.unique(),'structureID',subcortical_structural_measures,95,'emd',True,False,'',groups[0],data_dir+'/references','subcortical_structural_reference_'+groups[0])

## compute reference dataset for diffusion b0 SNR - CC
diffusion_snr_measures = ['snr']
diffusion_snr_distances, diffusion_snr_outliers, diffusion_snr_reference_dataframe, diffusion_snr_reference_json = outlierDetection(diffusion_snr_data,['b0'],'structureID',diffusion_snr_measures,95,'emd',True,False,'',groups[0],data_dir+'/references','diffusion_snr_callosal_reference_'+groups[0])

# merge reference datasets together for easier download and upload
mergeStructuralDiffusionJson(cortical,data_dir+'/references/cortical_structural_reference_'+groups[0],data_dir+'/references/cortical_diffusion_reference_'+groups[0],data_dir+'/references/cortical_reference_'+groups[0])
mergeStructuralDiffusionJson(subcortical,data_dir+'/references/subcortical_structural_reference_'+groups[0],data_dir+'/references/subcortical_diffusion_reference_'+groups[0],data_dir+'/references/subcortical_reference_'+groups[0])
mergeStructuralDiffusionJson(track_data,data_dir+'/references/tractmeasures_diffusion_reference_'+groups[0]+'_tensor',data_dir+'/references/tractmeasures_diffusion_reference_'+groups[0]+'_noddi',data_dir+'/references/tractmeasures_diffusion_reference_'+groups[0])
mergeStructuralDiffusionJson(track_data,data_dir+'/references/tractmeasures_structural_reference_'+groups[0],data_dir+'/references/tractmeasures_diffusion_reference_'+groups[0],data_dir+'/references/tractmeasures_reference_'+groups[0])

print("creating references complete")

creating references for subcortical cortical and tractmeasures data
anterioFrontalCC
forcepsMajor
forcepsMinor
leftAnterioFrontoCerebellar
leftArc
leftAslant
leftCST
leftContraAnterioFrontoCerebellar
leftContraMotorCerebellar
leftIFOF
leftILF
leftMDLFang
leftMDLFspl
leftMotorCerebellar
leftOccipitoCerebellar
leftParietoCerebellar
leftSLF1And2
leftSLF3
leftTPC
leftThalamicoCerebellar
leftUncinate
leftVOF
leftbaum
leftcingulum
leftfrontoThalamic
leftmeyer
leftmotorThalamic
leftpArc
leftparietoThalamic
leftspinoThalamic
lefttemporoThalamic
middleFrontalCC
parietalCC
rightAnterioFrontoCerebellar
rightArc
rightAslant
rightCST
rightContraAnterioFrontoCerebellar
rightContraMotorCerebellar
rightIFOF
rightILF
rightMDLFang
rightMDLFspl
rightMotorCerebellar
rightOccipitoCerebellar
rightParietoCerebellar
rightSLF1And2
rightSLF3
rightTPC
rightThalamicoCerebellar
rightUncinate
rightVOF
rightbaum
rightcingulum
rightfrontoThalamic
rightmeyer
rightmotorThalamic
rightpArc
rightparietoThalamic
rightspino

In [ ]:
#### generate figures documenting outlier detection capabilities
## track data: example anterior frontal CC
track_diffusion_reference_dataframe = pd.read_csv(data_dir+'/references/tractmeasures_reference_'+groups[0]+'.csv')
track_diffusion_reference_dataframe['classID'] = [ 'abcd' for f in track_diffusion_reference_dataframe['subjectID'] ]
track_diffusion_reference_dataframe['colors'] = [ 'green' for f in track_diffusion_reference_dataframe['subjectID'] ]

# generate plot pre-outlier computation
fig = plotProfiles(['anterioFrontalCC'],track_data,['fa'],'mean','std',"","")
plt.savefig('./img/anterior_frontal_cc_fa_pre_outliers_example.eps')
plt.savefig('./img/anterior_frontal_cc_fa_pre_outliers_example.png')
plt.close()

# generate outliers and reference plot
fig = plotProfiles(['anterioFrontalCC'],track_diffusion_reference_dataframe.loc[track_diffusion_reference_dataframe['structureID'] == 'anterioFrontalCC'],['fa'],'mean','std',"","")
sns.lineplot(x='nodeID',y='fa',data=track_data.loc[track_data['structureID'] == 'anterioFrontalCC'].loc[~track_data['subjectID'].isin(track_diffusion_reference_dataframe.loc[track_diffusion_reference_dataframe['structureID'] == 'anterioFrontalCC'].loc[~track_diffusion_reference_dataframe['fa'].isna()]['subjectID'].tolist())],hue='subjectID',palette=colors_dict,legend=False,linewidth=5)
plt.savefig('./img/anterior_frontal_cc_fa_outliers_example.eps')
plt.savefig('./img/anterior_frontal_cc_fa_outliers_example.png')
plt.close()

# generate reference plot
fig = plotProfiles(['anterioFrontalCC'],track_diffusion_reference_dataframe.loc[track_diffusion_reference_dataframe['structureID'] == 'anterioFrontalCC'].loc[~track_diffusion_reference_dataframe['fa'].isna()],['fa'],'mean','std',img_dir,'anterior')
plt.savefig('./img/anterior_frontal_cc_fa_reference_example.eps')
plt.savefig('./img/anterior_frontal_cc_fa_reference_example.png')
plt.close()

## subcortical data: left hippocampus
subcortical_structural_reference_dataframe = pd.read_csv(data_dir+'/references/subcortical_reference_'+groups[0]+'.csv')

# generate distribution of volume pre outliers
sns.violinplot(x='volume',data=subcortical.loc[subcortical['structureID'] == 'Left-Hippocampus'],scale='count',inner='points')
plt.savefig('./img/left_hippocampus_volume_pre_outliers_example.png')
plt.savefig('./img/left_hippocampus_volume_pre_outliers_example.eps')
plt.close()

# generate distribution of volume with outliers
sns.violinplot(x='volume',data=subcortical_structural_reference_dataframe.loc[subcortical_structural_reference_dataframe['structureID'] == 'Left-Hippocampus'].loc[~subcortical_structural_reference_dataframe['volume'].isna()],scale='count',inner='points')
sns.swarmplot(x='volume',data=subcortical.loc[subcortical['structureID'] == 'Left-Hippocampus'].loc[~subcortical['subjectID'].isin(subcortical_structural_reference_dataframe.loc[subcortical_structural_reference_dataframe['structureID'] == 'Left-Hippocampus'].loc[~subcortical_structural_reference_dataframe['volume'].isna()]['subjectID'].tolist())],color='red')
plt.savefig('./img/left_hippocampus_volume_outliers_example.png')
plt.savefig('./img/left_hippocampus_volume_outliers_example.eps')
plt.close()

# generate reference
sns.violinplot(x='volume',data=subcortical_structural_reference_dataframe.loc[subcortical_structural_reference_dataframe['structureID'] == 'Left-Hippocampus'].loc[~subcortical_structural_reference_dataframe['volume'].isna()],scale='count',inner='points')
plt.savefig('./img/left_hippocampus_volume_reference_example.png')
plt.savefig('./img/left_hippocampus_volume_reference_example.eps')
plt.close()

## fmri networks - average degree: DMN
grab default mode a network nodes and average
if os.path.isfile(data_dir+'/'+groups[0]+'-within-network-default-a-preclean-fc.csv'):
    default_a_preclean_df = pd.read_csv(data_dir+'/'+groups[0]+'-within-network-default-a-preclean-fc.csv')
else:
    default_a_preclean_df = fmri_yeo_networks_adjacency_mean_fc.loc[fmri_yeo_networks_adjacency_mean_fc['structureID'] == 'DefaultA']
    default_a_preclean_df.to_csv(data_dir+'/'+groups[0]+'-within-network-default-a-preclean-fc.csv',index=False)

# generate distribution of volume pre outliers
sns.violinplot(x='FC',data=default_a_preclean_df,scale='count',inner='points')
plt.savefig('./img/default_a_fc_pre_outliers_example.png')
plt.savefig('./img/default_a_fc_pre_outliers_example.eps')
plt.close()

# generate outliers and reference plot
default_a_distances, default_a_outliers, default_a_reference_dataframe, default_a_reference_json = outlierDetection(default_a_preclean_df,default_a_preclean_df.structureID.unique(),'structureID',['FC'],95,'emd',True,False,"",groups[0],data_dir+'/references','default_a_fc_references_'+groups[0])

# generate distribution of node degree with outliers
outliers_subs = default_a_outliers['subjectID'].unique()
default_a_reference = default_a_preclean_df.loc[~default_a_preclean_df['subjectID'].isin(outliers_subs)]
default_a_outliers = default_a_preclean_df.loc[default_a_preclean_df['subjectID'].isin(outliers_subs)]
sns.violinplot(x='FC',data=default_a_reference,scale='count',inner='points')
sns.swarmplot(x='FC',data=default_a_outliers,color='red')
plt.savefig('./img/default_a_fc_outliers_example.png')
plt.savefig('./img/default_a_fc_outliers_example.eps')
plt.close()

# generate reference
sns.violinplot(x='FC',data=default_a_reference,scale='count',inner='points')
plt.savefig('./img/default_a_fc_reference_example.png')
plt.savefig('./img/default_a_fc_reference_example.eps')
plt.close()

In [ ]:
## replicate Fukutomi et al 2018 results
# identify glasser structures with average snr > 17
good_glasser_structures = [ f for f in cortical['structureID'].unique()[cortical.groupby('structureID',as_index=False).mean()['snr'] > 17] if not 'lh' in f if not 'rh' in f]

# grab reference datasets
cortical_ref = pd.read_csv(data_dir+'/references/cortical_reference_'+groups[0]+'.csv')

# extract only the good structures
cortical_ref_good_snr = cortical_ref.loc[cortical_ref['structureID'].isin(good_glasser_structures)]

# plot data
singleplotScatter("",cortical_ref_good_snr.groupby(['structureID'],as_index=False).mean(),cortical_ref_good_snr.groupby(['structureID'],as_index=False).mean(),'average_thickness_mm','odi',False,'structureID','structureID','ravel','linreg',False,True,img_dir,"study_replication_cortex_tissue_mapping_"+groups[0]+"_structure_average")